In [1]:
# To read data from diamonds.csv
import pandas as pd
import numpy as np
import scipy
import math
from sklearn.model_selection import train_test_split
headers = ["carat","cut","color","clarity","depth","table","price","x","y","z"]
data = pd.read_csv('diamonds.csv', na_values='?',    
         header=None,  names = headers) 
data = data.reset_index(drop=True)
data = data.iloc[1:]
data.describe()
#print(data) 

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940,53940,53940,53940,53940,53940,53940,53940,53940,53940
unique,273,5,7,8,184,127,11602,554,552,375
top,0.3,Ideal,G,SI1,62,56,605,4.37,4.34,2.7
freq,2604,21551,11292,13065,2239,9881,132,448,437,767


In [2]:
# Defining x and y
x = data.drop(['price'], axis=1)
y = data['price']

#x.shape, y.shape
x = x.drop(x[x["x"]==0].index)
x = x.drop(x[x["y"]==0].index)
x = x.drop(x[x["z"]==0].index)

#Encoding Categorical Data 
handle_cut = {'cut': {'Fair':1 ,'Good':2 ,'Very Good':3 ,'Premium': 4,'Ideal': 5 } }
handle_color = {'color': {'J':1 ,'I':2 ,'H':3 ,'G': 4 ,'F':5 ,'E':6 ,'D':7 } }
handle_clarity = {'clarity': {'I1':1 ,'SI2':2 ,'SI1':3 ,'VS2':4 ,'VS1':5 ,'VVS2':6 ,'VVS1':7 ,'IF':8 } }
x.replace(handle_cut, inplace=True)
x.replace(handle_color, inplace=True)
x.replace(handle_clarity, inplace=True)

x = x.astype(float)
x = x.to_numpy()
y = y.astype(float)
y = y.to_numpy()

#Modifing x to 1d array
x_modified=np.delete(x,1,1)
x_modified=np.delete(x_modified,1,1)
x_modified=np.delete(x_modified,1,1)
x_modified=np.delete(x_modified,1,1)
x_modified=np.delete(x_modified,1,1)

In [7]:
#Correlation between features and the Diamond Price

corr_carat = scipy.stats.spearmanr(y,x[:,0])
corr_cut = scipy.stats.spearmanr(y,x[:,1])
corr_color = scipy.stats.spearmanr(y,x[:,2])
corr_clarity = scipy.stats.spearmanr(y,x[:,3])
corr_depth = scipy.stats.spearmanr(y,x[:,4])
corr_table = scipy.stats.spearmanr(y,x[:,5])
corr_x = scipy.stats.spearmanr(y,x[:,6])
corr_y = scipy.stats.spearmanr(y,x[:,7])
corr_z = scipy.stats.spearmanr(y,x[:,8])

print("carat:" + str(corr_carat))
print("cut:" + str(corr_cut))
print("color:" + str(corr_color))
print("clarity:" + str(corr_clarity))
print("depth:" + str(corr_depth))
print("table:" + str(corr_table))
print("x:" + str(corr_x))
print("y:" + str(corr_y))
print("z:" + str(corr_z))

carat:SpearmanrResult(correlation=0.9628827988813001, pvalue=0.0)
cut:SpearmanrResult(correlation=-0.09297484475554442, pvalue=7.586662851897269e-104)
color:SpearmanrResult(correlation=-0.15014215342958673, pvalue=2.0275054916076164e-269)
clarity:SpearmanrResult(correlation=-0.21152748395684604, pvalue=0.0)
depth:SpearmanrResult(correlation=0.01001966963455154, pvalue=0.019961465178476374)
table:SpearmanrResult(correlation=0.17178447793764812, pvalue=0.0)
x:SpearmanrResult(correlation=0.9631961087113347, pvalue=0.0)
y:SpearmanrResult(correlation=0.9627188193892315, pvalue=0.0)
z:SpearmanrResult(correlation=0.9572322686120687, pvalue=0.0)


In [3]:
#splitting
train_x,test_x,train_y,test_y = train_test_split(x_modified,y, test_size=0.2, random_state=1)

#modelling
train_x = np.append(np.ones([train_x.shape[0],1]),train_x, axis=1)
# Calculating coef by using W = (A.T * A)^(-1) * A.T * Y
coef = np.dot(np.dot(np.linalg.inv(np.dot(train_x.T,train_x)),train_x.T),train_y)

#Calculating y = W.T * X + b
pred = np.sum((test_x*coef[1:]),axis=1)+coef[0]

In [6]:
#Defining Rsquared
def rsquared(pred,y):
    corr_matrix = np.corrcoef(pred,y)
    corr = corr_matrix[0,1]
    rsq=corr**2
    return rsq

#Calulating Errors
rsquared_error = rsquared(pred,test_y)
mae_error = np.sum(abs(pred - test_y))/test_y.shape[0]
mse_error = np.sum((pred - test_y)**2)/test_y.shape[0]

print("rsquared_error: " + str(rsquared_error) + "    mae_error: " + str(mae_error) + "    mse_error: " + str(mse_error))

rsquared_error: 0.8547459446154654    mae_error: 912.7301120437912    mse_error: 2257937.9498545392
